In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


347 album were sold around the word

In [3]:
%%sql
select count(*) as num_album,  ROUND(count(*) * 100.0 /
                                         (select count(*)
  
                                          from album),2)  as Percentage
from album


 * sqlite:///chinook.db
Done.


num_album,Percentage
347,100.0


Number of track sold around the world by genre

In [4]:
%%sql
SELECT genre.name AS genre, COUNT(track.track_id) AS num_of_track,

ROUND(count(track.track_id) * 100.0 /
            (select count(*) from track),2)  as Percentage
  FROM  genre
  JOIN  track
   ON  genre.genre_id = track.genre_id
  JOIN album
   ON track.album_id =album.album_id
 GROUP BY genre.name
ORDER by num_of_track DESC
 LIMIT 5



 * sqlite:///chinook.db
Done.


genre,num_of_track,Percentage
Rock,1297,37.03
Latin,579,16.53
Metal,374,10.68
Alternative & Punk,332,9.48
Jazz,130,3.71


 Query show number tracked sold  It gives also the genre of the track. ( in Uk )

In [5]:

%%sql

SELECT genre.name AS genre, COUNT(invoice_line.track_id) AS num_of_track, 

/*ROUND(count() * 100.0 /
            (select count(*) from invoice_line),2)  as Percentage*/


ROUND(SUM(invoice_line.quantity * invoice_line.unit_price *100.0)/
             (SELECT SUM(total)
                FROM invoice WHERE billing_country is 'United Kingdom'),2) AS sales_pourc

  FROM  genre
  JOIN  track
   ON  genre.genre_id = track.genre_id
  JOIN album
   ON track.album_id =album.album_id
JOIN invoice_line
ON track.track_id = invoice_line.track_id
 JOIN invoice
ON invoice_line.invoice_id = invoice.invoice_id 
WHERE invoice.billing_country in  ( 'United Kingdom')
GROUP BY genre.name
ORDER by num_of_track DESC
 LIMIT 5

 * sqlite:///chinook.db
Done.


genre,num_of_track,sales_pourc
Rock,166,66.94
Metal,31,12.5
Alternative & Punk,24,9.68
Latin,7,2.82
Classical,6,2.42


 Query show number album  It gives also the percentage overall 

In [6]:
%%sql

SELECT 
   genre.name AS genre, album.title as album, count(album.album_id) as num_of_album , artist.name as artist,
    
    ROUND(count(*) * 100.0 /
            (select count(*) from album),2 )  as Percentage
    
 FROM  genre
 JOIN  track
   ON  genre.genre_id = track.genre_id     
 JOIN invoice_line as i_l
   ON i_l.track_id = track.track_id
 JOIN invoice
   ON i_l.invoice_id = invoice.invoice_id
 JOIN album
   ON track.album_id = album.album_id
 JOIN artist
   ON album.artist_id = artist.artist_id
WHERE invoice.billing_country in  ( 'United Kingdom')
GROUP BY album.album_id
ORDER BY num_of_album DESC
LIMIT 5

 * sqlite:///chinook.db
Done.


genre,album,num_of_album,artist,Percentage
Rock,For Those About To Rock We Salute You,20,AC/DC,5.76
Metal,Ace Of Spades,17,Motörhead,4.9
Rock,The Police Greatest Hits,16,The Police,4.61
Rock,By The Way,16,Red Hot Chili Peppers,4.61
Rock,Jagged Little Pill,14,Alanis Morissette,4.03


Employee sales performance in UK

In [7]:
%%sql
SELECT  invoice.billing_country, employee.last_name || ' ' || employee.first_name AS Employee, 
SUBSTRING(employee.hire_date, 1 , 10) AS Hire_date, employee.country,
ROUND(SUM(invoice_line.quantity * invoice_line.unit_price),2)  AS total_sales,
SUM(invoice_line.quantity) AS quantity_sold,
ROUND(SUM(invoice_line.quantity * invoice_line.unit_price *100.0)/
             (SELECT SUM(total)
                FROM invoice   WHERE invoice.billing_country in ( 'United Kingdom')  ),2) AS sales_prop
 FROM employee
 JOIN customer
   ON employee.employee_id = customer.support_rep_id
 JOIN invoice
   ON invoice.customer_id = customer.customer_id
 JOIN invoice_line
   ON invoice.invoice_id = invoice_line.invoice_id
WHERE invoice.billing_country in ( 'United Kingdom')
GROUP BY employee.employee_id
ORDER BY ROUND(SUM(total),2) DESC


 * sqlite:///chinook.db
Done.


billing_country,Employee,Hire_date,country,total_sales,quantity_sold,sales_prop
United Kingdom,Peacock Jane,2017-04-01,Canada,166.32,168,67.74
United Kingdom,Johnson Steve,2017-10-17,Canada,79.2,80,32.26


In [8]:
%%sql
SELECT SUM(total) as total
  FROM invoice
 WHERE invoice.billing_country in ( 'United Kingdom')

 * sqlite:///chinook.db
Done.


total
245.52


The next query show the total of sale per country

In [9]:
%%sql
SELECT billing_country, 
 ROUND(SUM(invoice.total),2) as total,
       ROUND(COUNT(*)*100.0/
             (SELECT COUNT(*) 
                FROM invoice), 2)  AS sales_prop
  
FROM invoice
GROUP BY billing_country
ORDER BY sales_prop DESC
LIMIT 5

 * sqlite:///chinook.db
Done.


billing_country,total,sales_prop
USA,1040.49,21.34
Canada,535.59,12.38
Brazil,427.68,9.93
France,389.07,8.14
Germany,334.62,6.68


 query that will return number customers in each country. All countries with 1 
customer are just mentionned as other .  Look the table "teste" to see lis of countries an number of customers.  

In [10]:
%%sql
DROP VIEW IF EXISTS pays;
CREATE VIEW pays as 
     SELECT COUNT(distinct customer.customer_id as Customers, 
       CASE
            WHEN COUNT(distinct customer.customer_id) = 1 then 'other' else country
           
            END AS country, ROUND(SUM(total),2) as total , COUNT(distinct invoice_id) as numb_ord
FROM customer  
JOIN invoice
  ON customer.customer_id = invoice.customer_id

GROUP BY country 
ORDER BY customers DESC;

SELECT * FROM pays
 LIMIT 5



 * sqlite:///chinook.db
Done.
(sqlite3.OperationalError) near "as": syntax error
[SQL: CREATE VIEW pays as 
     SELECT COUNT(distinct customer.customer_id as Customers, 
       CASE
            WHEN COUNT(distinct customer.customer_id) = 1 then 'other' else country
           
            END AS country, ROUND(SUM(total),2) as total , COUNT(distinct invoice_id) as numb_ord
FROM customer  
JOIN invoice
  ON customer.customer_id = invoice.customer_id

GROUP BY country 
ORDER BY customers DESC;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [11]:
%%sql
DROP VIEW IF EXISTS others;
CREATE VIEW others as
     SELECT COUNT(*) as customers, country,  SUM(total) as total, SUM(numb_ord) as numb_ord
FROM pays
WHERE country = 'other';

SELECT * FROM others



 * sqlite:///chinook.db
Done.
Done.
(sqlite3.OperationalError) no such table: main.pays
[SQL: SELECT * FROM others]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [12]:
%%sql
DROP VIEW IF EXISTS countries;
CREATE VIEW VIEW countries as
          SELECT * from pays
           WHERE country != 'other'
   UNION ALL
SELECT *
  FROM others;

SELECT * FROM countries
 LIMIT 5

 * sqlite:///chinook.db
Done.
(sqlite3.OperationalError) near "countries": syntax error
[SQL: CREATE VIEW VIEW countries as
          SELECT * from pays
           WHERE country != 'other'
   UNION ALL
SELECT *
  FROM others;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


The query below is a test. it shows the number of order by country

In [13]:
%%sql
SELECT billing_country, COUNT(distinct invoice_id) as numb_ord
  FROM invoice

GROUP BY billing_country
ORDER BY numb_ord desc 
LIMIT 5

 * sqlite:///chinook.db
Done.


billing_country,numb_ord
USA,131
Canada,76
Brazil,61
France,50
Germany,41


In [14]:
%%sql
SELECT * , round((total / customers),2) as cust_spent  , round((total / numb_ord),2) as Avg_ord
  FROM countries
GROUP BY country
ORDER BY avg_ord DESC , cust_spent DESC, numb_ord DESC


 * sqlite:///chinook.db
(sqlite3.OperationalError) no such table: countries
[SQL: SELECT * , round((total / customers),2) as cust_spent  , round((total / numb_ord),2) as Avg_ord
  FROM countries
GROUP BY country
ORDER BY avg_ord DESC , cust_spent DESC, numb_ord DESC]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


The query above shows number of customer per country.
Notice where country = 'other' this is all the countries that have one customer only. So 15 of them
cust_spent represent how much in specific country a customer spend. In Canda it is  £ 66.95 (535.59 / 8 = 85.54)
Avg_ord represent how much was spent on average per item. In india it is 8.72. £ 183.15 for 21 items

based on those record Csech Republic is top of list as each customer spent at list £ 9.11
UK has 8.77.  
-- One thing to do can be increase the number of items bought by those countries 
-- or reduce by few pences the prices to increase the quantity sold

Czech Republic, UK and India have few customers but have a lot order. 
Thing to consider is to try have more customers

In [15]:
%%sql
DROP VIEW IF EXISTS mm;


 * sqlite:///chinook.db
Done.


[]